In [ ]:
import numpy as np
from scipy.stats import norm
import copy
import matplotlib.pyplot as plt
from __future__ import division

In [1]:
from PD_models import agent_based_model

In [ ]:
class agent_based_model:
    
    """
    functions: 
        normal_alpha_syn_growth_region: growth of normal alpha-syn in regions
        normal_alpha_syn_growth_edge: mobility patterns of normal alpha-syn in edges
        normal_alpha_syn_growth_edge_discrete: a discrete version of mobility patterns, using length rather than probability to control the exits and enters
        inject_mis: inject misfolded alpha-syn into seed region. default: inject one misfolded alpha-syn into substantia nigra
        misfolded_alpha_syn_spread_region: interactions of normal and misfolded alpha-syn
        misfolded_alpha_syn_spread_edge: mobility patterns of normal/misfolded alpha-syn in edges
        misfolded_alpha_syn_spread_edge_discrete: a discrete version....
        record_to_history: record the misfolded/normal alpha-syn population at each time step
    usage:
        ref_model = agent_based_model(sconn_len=sconn_len, sconn_den=sconn_den, snca=snca, gba=gba, roi_size=roi_size, fconn=fconn, fcscale=0)
        T_total = 10000
        for t in range(T_total):
            ref_model.record_to_history()
            ref_model.normal_alpha_syn_growth_edge()
            ref_model.normal_alpha_syn_growth_region()
        ref_model.inject_mis()
        for t in range(T_total):
            ref_model.record_to_history()
            ref_model.misfolded_alpha_syn_spread_edge()
            ref_model.misfolded_alpha_syn_spread_region()
        print(ref_model.mis)"""
    
    # constructor
    def __init__(self, v=1 , N_regions=42, dt=0.01, sconn_len=None, sconn_den=None, snca=None, gba=None, roi_size=None, fconn=None, fcscale=None):
        
        # number of regions
        self.N_regions = N_regions
        
        # store number of normal and misfolded proteins in regions
        self.nor,self.mis = [np.zeros((N_regions,), dtype = np.int)] * 2
        self.nor_history, self.mis_history = [np.empty((0, N_regions), dtype = np.int)] * 2        
        
        # store number of normal and misfolded proteins in paths
        self.sconn_len = np.int_(np.round(sconn_len / v))
        (self.idx_x, self.idx_y) = np.nonzero(self.sconn_len)
        self.non_zero_lengths = self.sconn_len[self.idx_x, self.idx_y]  # int64
        self.path_nor, self.path_mis = [ [ [[] for y in range(N_regions)] for x in range(N_regions)] ] * 2
        
        #### is there more efficient way to do this?  --- to be updated.......
        for x, y, v in zip(self.idx_x, self.idx_y, self.non_zero_lengths):
            self.path_nor[x][y], self.path_mis[x][y] = [ [0 for k in range(v)] ] * 2
            
        # record the trajectory
        self.path_nor_history = []
        self.path_mis_history = []
        
        # continuous path and path history
        self.path_nor_cont, self.path_mis_cont = [np.zeros((N_regions, N_regions), dtype = np.int)] * 2
        self.path_nor_cont_history, self.path_mis_cont_history = [np.empty((0, self.N_regions, self.N_regions))] * 2
        
        # time step
        self.dt = dt
        
        # synthesis rate and clearance rate
        self.synthesis_rate = norm.cdf(snca) * self.dt
        self.clearance_rate = 1 - np.exp(-norm.cdf(gba) * self.dt)       
        
        # probability of exit a path is set to v/sconn_len
        sconn_len[sconn_len ==0] = np.inf
        self.prob_exit = self.dt * v / sconn_len
        self.prob_exit[sconn_len==0] = 0 # remove NaNs....
        
        # travel weights
        self.weights = np.exp(fcscale * fconn) * sconn_den
        
        self.weights = np.sum(self.weights, axis = 1) * np.eye(self.N_regions) + self.weights
        
        # scale 
        self.weights = self.weights / np.sum(self.weights, axis = 1).reshape(self.N_regions, 1)
        
        self.weights_euler = self.weights * self.dt
        self.weights_euler[np.eye(self.N_regions)==1] = 0
        self.weights_euler[np.eye(self.N_regions)==1] = 1-np.sum(self.weights_euler, axis=1)
        
        # region size
        self.roi_size = roi_size.flatten()
        
        self.synthesis_control = np.int_(roi_size.flatten())
        
                
    def normal_alpha_syn_growth_region(self):
        """step: normal alpha-syn synthesized and cleared in regions"""

        self.nor -= np.array([np.sum(np.random.uniform(0, 1, (v, )) < k) 
                             for k, v in zip(self.clearance_rate, self.nor)])
        
        
        ## synthesis
        self.nor += np.array([np.sum(np.random.uniform(0, 1, (v, )) < k) 
                              for k, v in zip(self.synthesis_rate, self.synthesis_control) ])
                              # or self.roi_size)])
    
    def normal_alpha_syn_growth_edge_discrete(self):
        """proteins are moving discretely in edges"""
        # alpha syn  -- from region to path
        # exit region
        exit_process = np.array([ np.random.multinomial(self.nor[k], self.weights[k]) 
                                 for k in range(self.N_regions)], dtype = np.int)
        
        # alpha syn -- from path to region
        # enter region
        enter_process = np.zeros((self.N_regions, self.N_regions), dtype=np.int)
        
        for x, y in zip(self.idx_x, self.idx_y):
            # fetch then remove the last element
            enter_process[x, y] = self.path_nor[x][y].pop()
            # update paths
            self.path_nor[x][y].insert(0, exit_process[x, y])

        # update regions 
        self.nor = exit_process[np.nonzero(np.eye(self.N_regions))] + np.sum(enter_process, axis = 0)
        
    def normal_alpha_syn_growth_edge(self):
        """proteins are moving contiously in edges"""
        
        # exit regions:
        exit_process = np.array([ np.random.multinomial(self.nor[k], self.weights_euler[k]) 
                                 for k in range(self.N_regions)], dtype = np.int) 
        exit_process[np.eye(self.N_regions)==1] = 0 # remove diag
        
        # enter regions:
        enter_process = np.zeros((self.N_regions, self.N_regions), dtype = np.int)
        
        for x, y in zip(self.idx_x, self.idx_y):
            enter_process[x, y] = np.sum(np.random.uniform(0, 1, (self.path_nor_cont[x, y], )) < self.prob_exit[x, y]*self.dt)
        
        # update:
        self.path_nor_cont += (exit_process - enter_process)        
        self.nor += (np.sum(enter_process, axis = 0) - np.sum(exit_process, axis=1) )
        
        
    def inject_mis(self, seed=41, initial_number=1):
        
        """inject initual_number misfolded protein into seed region"""
        # initial_number must be an interger
        self.mis[seed] = initial_number
        #print('inject %d misfolded alpha-syn into region %d' % (initial_number, seed))
        
    
    def misfolded_alpha_syn_spread_edge_discrete(self):
        """ step in paths for normal and misfolded alpha syn"""
        
        ############## misfolded alpha synuclein ###########
        # exit regions
        exit_process = np.array([ np.random.multinomial(v, self.weights[k]) 
                                 for k, v in enumerate(self.mis)], dtype = np.int)
        # alpha syn -- from path to region
        # enter region
        enter_process = np.zeros((self.N_regions, self.N_regions), dtype = np.int)
        for x, y in zip(self.idx_x, self.idx_y):
            # fetch then remove the last element
            enter_process[x, y] = self.path_mis[x][y].pop()
            # update paths
            self.path_mis[x][y].insert(0, exit_process[x, y])

        # update regions 
        self.mis = exit_process[np.nonzero(np.eye(self.N_regions))] + np.sum(enter_process, axis = 0)
        
        ########### for the normal alpha syuclein ###########
        exit_process = np.array([ np.random.multinomial(self.nor[k], self.weights[k]) 
                                 for k in range(self.N_regions)], dtype = np.int)
        
        # alpha syn -- from path to region
        # enter region
        enter_process = np.zeros((self.N_regions, self.N_regions), dtype=np.int)
        
        for x, y in zip(self.idx_x, self.idx_y):
            # fetch then remove the last element
            enter_process[x, y] = self.path_nor[x][y].pop()
            # update paths
            self.path_nor[x][y].insert(0, exit_process[x, y])

        # update regions 
        self.nor = exit_process[np.nonzero(np.eye(self.N_regions))] + np.sum(enter_process, axis = 0)
        
    def misfolded_alpha_syn_spread_edge(self):
        """proteins are moving continously"""
        ##### misfolded alpha synuclein #####
        exit_process = np.array([ np.random.multinomial(v, self.weights_euler[k]) 
                                 for k, v in enumerate(self.mis)], dtype = np.int) 
        exit_process[np.eye(self.N_regions)==1] = 0
        
        enter_process = np.zeros((self.N_regions, self.N_regions), dtype = np.int)
        # enter regions:
        for x, y in zip(self.idx_x, self.idx_y):
            enter_process[x, y] = np.sum(np.random.uniform(0, 1, (self.path_mis_cont[x, y], ))< self.prob_exit[x, y] * self.dt)
        
        # update
        self.path_mis_cont += (exit_process - enter_process)
        self.mis +=  (np.sum(enter_process, axis = 0) - np.sum(exit_process, axis = 1) )
        
        ####### normal alpha synuclein #######
        # exit regions:
        exit_process = np.array([ np.random.multinomial(self.nor[k], self.weights_euler[k]) 
                                 for k in range(self.N_regions)], dtype = np.int) * self.dt
        exit_process[np.eye(self.N_regions)==1] = 0
        
        # enter regions:
        enter_process = np.zeros((self.N_regions, self.N_regions), dtype = np.int)
        for x, y in zip(self.idx_x, self.idx_y):
            enter_process[x, y] = np.sum(np.random.uniform(0, 1, (self.path_nor_cont[x, y], ) ) < self.prob_exit[x, y]*self.dt)
            
        # update:
        self.path_nor_cont += (exit_process - enter_process)
        self.nor += (np.sum(enter_process, axis = 0) - np.sum(exit_process, axis=1) )
        
    def misfolded_alpha_syn_spread_region(self, trans_rate=1):
        """clearance and synthesis of normal/misfolded alpha-syn/ transsmssion process in regions"""
        ## clearance
        cleared_nor = np.array([np.sum(np.random.uniform(0, 1, (v, )) < k) 
                             for k, v in zip(self.clearance_rate, self.nor)])
        
        cleared_mis = np.array([np.sum(np.random.uniform(0, 1, (v, )) < k) 
                             for k, v in zip(self.clearance_rate, self.mis)])
        
        self.prob_infected = 1 - np.exp(- (self.dt * self.mis * trans_rate / (self.roi_size)) )
        # the remaining after clearance
        self.nor -= cleared_nor
        self.mis -= cleared_mis
        #self.prob_infected = 1 - np.exp(- (self.mis / self.roi_size) )
        infected_nor = np.array([np.sum(np.random.uniform(0, 1, (v, )) < k) 
                             for k, v in zip(self.prob_infected, self.nor)])
        # update self.nor and self.mis
        self.nor += (np.array([np.sum(np.random.uniform(0, 1, (v, )) < k) 
                              for k, v in zip(self.synthesis_rate, self.roi_size)]) - infected_nor)
        self.mis += infected_nor 
        
        #print(self.mis)
        
    def transmission_path(self, trans_rate_path):
        
        """transmission process in path (default shut down)"""
        for x, y, v in zip(self.idx_x, self.idx_y, self.non_zero_lengths):
            ### perhaps trans_rate_path should be set to 1/v ?
            # transmission rate is scaled by exp(distance) in voxel space
            path_nor_temp = np.array(self.path_nor[x][y])
            path_mis_temp = np.array(self.path_mis[x][y])
            rate_get_infected = (path_mis_temp * trans_rate_path) / np.exp(np.absolute(np.arange(v) - np.arange(v)[np.newaxis].T) )
            prob_get_infected = 1 - np.exp(np.sum(-rate_get_infected, axis = 1))
            infected_path = np.array([np.sum(np.random.uniform(0, 1, (k, ))<v) for 
                                      k, v in zip(self.path_nor[x][y], prob_get_infected)])
            
            # update self.path_nor and self.path_mis
            path_nor_temp -= infected_path
            path_mis_temp += infected_path
            
            self.path_nor[x][y] = path_nor_temp.tolist()
            self.path_mis[x][y] = path_mis_temp.tolist()
            
        
    def record_to_history_discrete(self):
        """record the results of each step into the recorder"""
        self.nor_history = np.append(self.nor_history, self.nor[np.newaxis], axis = 0)
        self.mis_history = np.append(self.mis_history, self.mis[np.newaxis], axis = 0)
        
        # record the mobility patterns in edges
        #self.path_nor_history.append(self.path_nor)
        #self.path_mis_history.append(self.path_mis)
        
    def record_to_history(self):
        """record the results of each step"""
        self.nor_history = np.append(self.nor_history, self.nor[np.newaxis], axis = 0)
        self.mis_history = np.append(self.mis_history, self.mis[np.newaxis], axis = 0)
        
        # record the mobility patterns in edges
        #self.path_nor_cont_history = np.append(self.path_nor_cont_history, self.path_nor_cont.reshape(1, 42, 42), axis = 0)
        #self.path_mis_cont_history = np.append(self.path_mis_cont_history, self.path_mis_cont.reshape(1, 42, 42), axis = 0)
        

In [ ]:
class SIR_model:
    """An SIR model to simulate the spread of alpha-syn"""
    
    # constructor
    def __init__(self, v=1 , N_regions=42, dt=0.01, sconn_len=None, sconn_den=None, snca=None, gba=None, roi_size=None, fconn=None, fcscale=None):
        
        # number of regions
        self.N_regions = N_regions
        
        # store number of normal and misfolded proteins in regions
        self.nor, self.mis = [np.zeros((N_regions, ))] * 2
        self.nor_history, self.mis_history = [np.empty((0, N_regions))] * 2
        
        # store number of normal and misfoded proteins in paths
        self.path_nor, self.path_mis = [np.zeros((N_regions, N_regions))] * 2
        self.path_nor_history, self.path_mis_history = [np.empty((0, N_regions, N_regions))] * 2
        
        # index of connected components
        (self.idx_x, self.idx_y) = np.nonzero(sconn_len)
        self.non_zero_lengths = sconn_len[np.nonzero(sconn_len)]
        
        # probability of exit a path is set to v/sconn_len
        sconn_len[sconn_len == 0] = np.inf
        self.prob_exit = v / sconn_len
        self.prob_exit[sconn_len==0] = 0
        
        # synthesis rate and  clearance rate
        self.synthesis_rate = norm.cdf(snca) * self.dt
        self.clearance_rate = 1 - np.exp( -norm.cdf(gba) * self.dt)
        
        # mobility pattern multinomial distribution
        self.weights = np.exp(fcscale * fconn) * sconn_den
        self.weights = np.diag(np.sum(sconn_den, axis = 1)) + sconn_den
        self.weights = self.weights / np.sum(self.weights, axis = 1)[np.newaxis].T
        
        # region size
        self.roi_size = roi_size.flatten() 
        
    def normal_alpha_syn_growth_region(self):
        """normal alpha-syn growing"""
        self.nor += (self.roi_size * self.synthesis_rate - self.nor * self.clearance_rate)
        
    def normal_alpha_syn_growth_edge(self):
        
        # enter paths
        enter_process = self.nor.reshape(self.N_regions, 1) * self.weights * self.dt
        enter_process[np.eye(self.N_regions)==1] = 0 # remove diag
        
        # exit paths
        exit_process = self.path_nor * self.prob_exit * self.dt
        
        # update paths and regions  
        self.nor += (np.sum(exit_process, axis = 0) - np.sum(enter_process, axis = 1) )
        self.path_nor += (enter_process - exit_process)        
        
    def inject_mis(self, seed=41, initial_number=1):
        """inject misfolded alpha-syn in seed region"""
        
        self.mis[seed] = initial_number
        #print('inject %d misfolded alpha-syn into region %d' % (initial_number, seed))
        
    def misfolded_alpha_syn_spread_edge(self):
        ##### misfolded alpha-syn
        # enter paths
        enter_process = self.mis.reshape(self.N_regions, 1) * self.weights * self.dt
        enter_process[np.eye(self.N_regions)==1] = 0 # remove diag
        
        # exit paths
        exit_process = self.path_mis * self.prob_exit
        
        # update paths and regions 
        self.mis += (-np.sum(enter_process, axis=1) + np.sum(exit_process, axis = 0))
        self.path_mis += (enter_process - exit_process)
        
        ###### normal alpha-syn
        # enter paths
        enter_process = self.nor.reshape(self.N_regions, 1) * self.weights * self.dt
        enter_process[np.eye(self.N_regions)==1] = 0 # remove diag
        
        # exit paths
        exit_process = self.path_nor * self.prob_exit * self.dt
        
        # update paths and regions  
        self.nor += (np.sum(exit_process, axis = 0) - np.sum(enter_process, axis = 1) )
        self.path_nor += (enter_process - exit_process)
        
    def misfolded_alpha_syn_spread_region(self, trans_rate=1):
        '"""the transmission process inside regions"""'
        
        prob_get_infected = 1 - np.exp(-self.mis * trans_rate * self.dt / self.roi_size)
        # clear process
        self.nor -= self.nor * self.clearance_rate
        self.mis -= self.mis * self.clearance_rate
        
        infected = self.nor * prob_get_infected
        self.nor += (self.roi_size * self.synthesis_rate - infected)
        self.mis += (infected)
        
    def transmission_path(self):
        
        """the transmission process in paths""" 
        pass
    
    def record_to_history(self):
        """record the results"""
        self.nor_history = np.append(self.nor_history, self.nor[np.newaxis], axis = 0)
        self.mis_history = np.append(self.mis_history, self.mis[np.newaxis], axis = 0)
        
        self.path_nor_history = np.append(self.path_nor_history, 
                                          self.path_nor.reshape(1, self.N_regions, self.N_regions), axis = 0)
        self.path_mis_history = np.append(self.path_mis_history, 
                                          self.path_mis.reshape(1, self.N_regions, self.N_regions), axis = 0)
        
        

In [ ]:
from scipy.io import loadmat
import numpy as np
#from scipy.stats.stats import zscore,pearsonr

In [ ]:
para = loadmat('/Users/yingqiuzheng/Desktop/Research/PD/Lh/DTIseeds/Resol1/parameters_final_normalized.mat')

In [ ]:
sconn_len = np.array(para['sc_len_total'])
sconn_den = np.array(para['sc_ncount_n_total'])
roi_size = np.array(para['ROIsize'])
gba = para['GBA']
snca = para['SNCA']
real_atrophy = para['real_atrophy']

In [ ]:
ref_model = agent_based_model(v=1 , N_regions=42, dt=0.01, sconn_len=sconn_len, sconn_den=sconn_den, snca=snca.flatten(), gba=gba.flatten(), roi_size=roi_size.flatten(), fconn=np.zeros((42, 42)), fcscale=0)

In [ ]:
for t in range(10000):
    ref_model.record_to_history()
    ref_model.normal_alpha_syn_growth_edge()
    ref_model.normal_alpha_syn_growth_region()
    print(ref_model.nor)    

In [ ]:
ratio = one_model.mis_history / (one_model.mis_history + one_model.nor_history)
plt.figure(figsize = (35, 5))
plt.imshow(ratio.T,  cmap=plt.cm.RdBu, aspect='auto')
plt.xlim(0, 6000)
plt.ylim(0, 41)
plt.colorbar()
plt.show()

In [ ]:
snca

In [3]:
np.arange(3)

NameError: name 'np' is not defined

In [4]:
print(ref_model.weights[:, 0])

NameError: name 'ref_model' is not defined

In [ ]:
for t in range(600):
    ref_model.record_to_history()
    ref_model.nor_step_region()
    ref_model.nor_travel()

In [ ]:
plt.plot(np.arange(42), (1  - np.exp(- 7 / ref_model.nor)  ) )
#plt.plot(np.arange(42), ref_model.clearance_rate)
plt.show()
#np.max((1  - np.exp(- 1 / roi_size.flatten())  )* ref_model.nor*ref_model.clearance_rate)

In [ ]:
plt.plot(np.arange(42), ref_model.nor_history[599,:])
plt.plot(np.arange(42), ref_model.synthesis_rate * ref_model.roi_size)
plt.show()

In [ ]:
plt.plot(ref_model.synthesis_rate * ref_model.roi_size, ref_model.nor_history[599,:], marker = 'o', ls='')
plt.show()

In [ ]:
print(norm.cdf(snca)/norm.cdf(gba))

In [ ]:
ref_model.nor

In [ ]:
one_model.prob_infected

In [ ]:
from scipy.io import loadmat
para2 = loadmat('/dagher/dagher6/yzheng/ICBM152_nlin_sym_09a/simulator_v3/workspace.mat')
nsconn_den = para2['nsconnDen_th']
np.sum(nsconn_den, axis = 1).shape
weights = nsconn_den / np.sum(nsconn_den, axis = 0)[np.newaxis]
print(np.sum(weights, axis = 0))

In [ ]:
ratio = one_model.mis_history / (one_model.mis_history + one_model.nor_history)
plt.imshow(ratio_cum.T, aspect='auto')
plt.xlim(0, 7000)
plt.ylim(0, 41)
plt.colorbar()
plt.show()

In [ ]:
weights = nsconn_den / np.sum(nsconn_den, axis = 0)[np.newaxis]
ratio_th = ratio.copy()
ratio_th[ratio<0] = 0
ratio_cum = np.dot( (1 - np.exp(-ratio_th)), weights)
ratio_cum = ratio_cum + 1-np.exp(-ratio_th)
ratio_cum = np.cumsum(ratio_cum[3000:], axis = 0)
(r, p) = pearsonr(ratio[-1, :-1], real_atrophy.flatten())
r

In [ ]:
plt.plot

In [ ]:
r

In [ ]:
plt.plot(np.arange(one_model.mis_history.shape[0]), one_model.mis_history[:, 41])
plt.show()

In [ ]:
ref_model.nor

In [ ]:
np.nonzero(sconn_len[41])

In [ ]:
sconn_len[41, np.nonzero(sconn_len[41])]

In [ ]:
r = np.zeros((42, 100))
for region in np.arange(42):
    """iterate through regions to find epicentre"""
    for simulation in np.arange(100):
        one_model = copy.deepcopy(ref_model)
        one_model.inject_mis(41, 1)
        for t in range(2000):
            one_model.record_to_history()
            one_model.transmission_region()
            one_model.nor_travel()
            one_model.mis_travel()
        
        ratio = one_model.mis_history / (one_model.mis_history + one_model.nor_history)
        ratio_th = ratio.copy()
        ratio_th[ratio<0.2] = 0
        ratio_cum = np.dot( (1 - np.exp(-ratio_th)), weights)
        ratio_cum = ratio_cum + 1-np.exp(-ratio_th)
        ratio_cum = np.cumsum(ratio_cum[600:], axis = 0)
        (r[region, simulation], p) = pearsonr(ratio_cum[-1, :-1], real_atrophy.flatten())
        if r[region,simulation] == np.nan:
            r[region, simulation] = 0

In [ ]:
r[np.isnan(r)] = 0

In [ ]:
mean_r = np.mean(r, axis = 1)

In [ ]:
sconn = loadmat('/dagher/dagher6/yzheng/ICBM152_nlin_sym_09a/sconn0712.mat')
sconn_len = sconn['sconnLen_expand']
sconn_den = sconn['sconnDen_expand']

In [ ]:
print(one_model.mis_history[:, 41])

In [ ]:
print(ref_model.synthesis_rate * roi_size.flatten())

In [ ]:
np.ones((3, ), dtype = np.int) * 0.01

In [ ]:
np.array([k for k in range(4)]) * 0.1

In [ ]:
?agent_based_model